# How to prepare the data

We use a simplified version of the data set constructed in Chapter 4, Alpha factor research. It consists of daily stock prices provided by Quandl for the 2010-2017 period and various engineered features. 
 
The decision tree models in this chapter are not equipped to handle missing or categorical variables, so we will apply dummy encoding to the latter after dropping any of the former.

In [30]:
%matplotlib inline
import warnings
import pandas as pd

import matplotlib.pyplot as plt

In [31]:
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

Depending on when you download the us stock metadata from NASDAQ, the number of shared tickers may change as NASDAQA regularly updates this information. As a consequence, the results you obtain when running the samples will be different.

In [32]:
with pd.HDFStore('../data/assets.h5') as store:
    prices = store['quandl/wiki/prices'].adj_close.unstack('ticker')
    stocks = store['us_equities/stocks']

In [33]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6834 entries, PIH to ZYME
Data columns (total 6 columns):
name         6834 non-null object
lastsale     6718 non-null float64
marketcap    5766 non-null float64
ipoyear      3038 non-null float64
sector       5288 non-null object
industry     5288 non-null object
dtypes: float64(3), object(3)
memory usage: 373.7+ KB


In [34]:
shared_tickers = prices.columns.intersection(stocks.index)
prices = prices.loc['2010': '2018', shared_tickers]
stocks = stocks.loc[shared_tickers, ['marketcap', 'ipoyear', 'sector']]

In [35]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2412 entries, A to ZUMZ
Data columns (total 3 columns):
marketcap    2407 non-null float64
ipoyear      1065 non-null float64
sector       2372 non-null object
dtypes: float64(2), object(1)
memory usage: 75.4+ KB


In [36]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2113 entries, 2010-01-04 to 2018-03-27
Columns: 2412 entries, A to ZUMZ
dtypes: float64(2412)
memory usage: 38.9 MB


### Remove outliers

In [37]:
returns = prices.resample('M').last().pct_change().stack().swaplevel()
returns.index.names = ['ticker', 'date']

In [38]:
returns = (returns[returns.between(left=returns.quantile(.05), 
                                   right=returns.quantile(.95))].to_frame('returns'))

In [39]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 200014 entries, (A, 2010-02-28 00:00:00) to (ZTS, 2018-03-31 00:00:00)
Data columns (total 1 columns):
returns    200014 non-null float64
dtypes: float64(1)
memory usage: 2.1+ MB


### Lagged returns

In [40]:
for t in range(1, 13):
    returns[f't-{t}'] = returns.groupby(level='ticker').returns.shift(t)
returns = returns.dropna()

### Time Period Dummies

In [41]:
dates = returns.index.get_level_values('date')
returns['year'] = dates.year
returns['month'] = dates.month
returns = pd.get_dummies(returns, columns=['year', 'month'])

In [42]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 171162 entries, (AAON, 2011-02-28 00:00:00) to (ZTS, 2018-03-31 00:00:00)
Data columns (total 33 columns):
returns      171162 non-null float64
t-1          171162 non-null float64
t-2          171162 non-null float64
t-3          171162 non-null float64
t-4          171162 non-null float64
t-5          171162 non-null float64
t-6          171162 non-null float64
t-7          171162 non-null float64
t-8          171162 non-null float64
t-9          171162 non-null float64
t-10         171162 non-null float64
t-11         171162 non-null float64
t-12         171162 non-null float64
year_2011    171162 non-null uint8
year_2012    171162 non-null uint8
year_2013    171162 non-null uint8
year_2014    171162 non-null uint8
year_2015    171162 non-null uint8
year_2016    171162 non-null uint8
year_2017    171162 non-null uint8
year_2018    171162 non-null uint8
month_1      171162 non-null uint8
month_2      171162 non-null uint8
month_3     

### Get stock characteristics

In [43]:
stocks['age'] = pd.qcut(stocks.ipoyear, q=5, labels=list(range(1, 6))).astype(float).fillna(0).astype(int)
stocks = stocks.drop('ipoyear', axis=1)

#### Create size proxy

In [44]:
stocks.marketcap.head()

ticker
A       1.960000e+10
AA      8.540000e+09
AAL     1.767000e+10
AAMC    1.104900e+08
AAN     3.200000e+09
Name: marketcap, dtype: float64

In [45]:
stocks['size'] = pd.qcut(stocks.marketcap, q=10, labels=list(range(1, 11)))
stocks = stocks.drop(['marketcap'], axis=1)

In [46]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2412 entries, A to ZUMZ
Data columns (total 3 columns):
sector    2372 non-null object
age       2412 non-null int64
size      2407 non-null category
dtypes: category(1), int64(1), object(1)
memory usage: 139.3+ KB


In [47]:
stocks = pd.get_dummies(stocks, 
                        columns=['size', 'age',  'sector'], 
                        prefix=['size', 'age', ''], 
                        prefix_sep=['_', '_', ''])
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2412 entries, A to ZUMZ
Data columns (total 28 columns):
size_1                   2412 non-null uint8
size_2                   2412 non-null uint8
size_3                   2412 non-null uint8
size_4                   2412 non-null uint8
size_5                   2412 non-null uint8
size_6                   2412 non-null uint8
size_7                   2412 non-null uint8
size_8                   2412 non-null uint8
size_9                   2412 non-null uint8
size_10                  2412 non-null uint8
age_0                    2412 non-null uint8
age_1                    2412 non-null uint8
age_2                    2412 non-null uint8
age_3                    2412 non-null uint8
age_4                    2412 non-null uint8
age_5                    2412 non-null uint8
Basic Industries         2412 non-null uint8
Capital Goods            2412 non-null uint8
Consumer Durables        2412 non-null uint8
Consumer Non-Durables    2412 non-null uint

### Combine data

In [48]:
data = (returns
        .reset_index('date')
        .merge(stocks, left_index=True, right_index=True)
        .dropna()
        .set_index('date', append=True))

s = len(returns.columns)
data.iloc[:, s:] = data.iloc[:, s:].astype(int).apply(pd.to_numeric, downcast='integer')
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 171162 entries, (A, 2011-03-31 00:00:00) to (ZUMZ, 2018-02-28 00:00:00)
Data columns (total 61 columns):
returns                  171162 non-null float64
t-1                      171162 non-null float64
t-2                      171162 non-null float64
t-3                      171162 non-null float64
t-4                      171162 non-null float64
t-5                      171162 non-null float64
t-6                      171162 non-null float64
t-7                      171162 non-null float64
t-8                      171162 non-null float64
t-9                      171162 non-null float64
t-10                     171162 non-null float64
t-11                     171162 non-null float64
t-12                     171162 non-null float64
year_2011                171162 non-null uint8
year_2012                171162 non-null uint8
year_2013                171162 non-null uint8
year_2014                171162 non-null uint8
year_2015           

### Store data

In [49]:
with pd.HDFStore('data.h5') as store:
    store.put('quandl/us/equities', data)